In [162]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [163]:
df_data_orig = pd.read_csv('data/mnist/train.csv')
df_data = df_data_orig.copy()
df_data.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
data = torch.tensor(df_data.to_numpy(), dtype=torch.float32)
X = data[:, 1:].reshape((-1, 1, 28, 28))
Y = data[:, 0].int()
Y_enc = torch.eye(10)[Y]

print(f"{X.shape=}")
print(f"{Y_enc.shape=}")

X.shape=torch.Size([42000, 1, 28, 28])
Y_enc.shape=torch.Size([42000, 10])


In [165]:
X_norm = X / 255

In [166]:
idx = torch.randperm(X_norm.shape[0])

X_norm = X_norm[idx]
Y_enc = Y_enc[idx]

n1 = int(0.8 * len(X))

X_train = X_norm[:n1]
Y_train = Y_enc[:n1]
X_val = X_norm[n1:]
Y_val = Y_enc[n1:]

X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_val = X_val.to(device)
Y_val = Y_val.to(device)

print(f"{X_train.shape=}")
print(f"{Y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{Y_val.shape=}")

X_train.shape=torch.Size([33600, 1, 28, 28])
Y_train.shape=torch.Size([33600, 10])
X_val.shape=torch.Size([8400, 1, 28, 28])
Y_val.shape=torch.Size([8400, 10])


In [167]:
class Model(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, (3, 3))    # (b, 8, 26, 26)
        self.mp1 = nn.MaxPool2d((2, 2))         # (b, 8, 13, 13)
        self.conv2 = nn.Conv2d(8, 16, (3, 3))   # (b, 16, 11, 11)
        self.mp2 = nn.MaxPool2d((2, 2))         # (b, 16, 5, 5)
        self.l1 = nn.Linear(16*5*5, 128)        # (b, 16*5*5)
        self.l2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.mp1(F.relu(self.conv1(x)))
        x = self.mp2(F.relu(self.conv2(x)))
        x = F.relu(self.l1(x.view(x.shape[0], -1)))
        return F.softmax(self.l2(x), 1)


In [168]:
model = Model()
m = model.to(device)

In [169]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [173]:
loss = None
batch_size = 128

for epoch in range(2000):
    optimizer.zero_grad()
    
    idx = torch.randperm(X_train.shape[0])
    x = X_train[idx]
    x = x[:batch_size]
    y = Y_train[idx]
    y = y[:batch_size]

    outputs = m(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

print(loss)

tensor(1.4659, device='cuda:0', grad_fn=<DivBackward1>)


In [174]:
output = model(X_val)
((output.max(1)[1] == Y_val.max(1)[1]).sum() / len(output))

tensor(0.9773, device='cuda:0')